# Importing Libraries

In [ ]:
import torch
from transformers import pipeline
from huggingface_hub import login
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM
import requests


In [ ]:
from google.colab import userdata
hf_token = userdata.get('hf_llama')
login(token=hf_token)

# Import Model

In [ ]:
# Initialize Llama model pipeline
model_id = "meta-llama/Llama-3.2-1B-Instruct"
llama_pipeline = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,  # Use float16 for better performance
    device_map="auto",  # Automatically use GPU if available
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


# Agent

In [ ]:
class UserPreferenceAgent:
    def __init__(self):
        self.preferences = {}

    def gather_preferences(self, user_input):
        """
        Takes user input and parses it to build preferences.
        """
        self.preferences = user_input
        return self.preferences



In [ ]:
# Model ID
model_id = "meta-llama/Llama-3.2-1B-Instruct"

class ItineraryAgent:
    def __init__(self, model_pipeline=None):
        """
        Initializes the itinerary agent with no API keys required.
        """
        self.model_pipeline = model_pipeline
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Load the model and move it to the device (CPU or CUDA)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"  # Correcting device reference
        self.model = AutoModelForCausalLM.from_pretrained(model_id).to(self.device)

    def fetch_weather(self, destination):
        """
        Fetches current weather for the destination from the free wttr.in API.
        """
        weather_url = f"https://wttr.in/{destination}?format=%C+%t"

        try:
            response = requests.get(weather_url)
            weather_data = response.text.strip()
            return weather_data
        except requests.exceptions.RequestException as e:
            return f"Error fetching weather data: {str(e)}"

    def generate_itinerary(self, preferences):
        """
        Generates an itinerary for the trip based on user preferences.
        """
        # Fetch weather data
        weather_data = self.fetch_weather(preferences['destination'])

        # Construct the prompt for itinerary generation using the LLM
        prompt = f"""
        Prompt: Give a breif description of the {preferences['destination']}.Maximum 2-5 lines.
        Generate a detailed itinerary for a {preferences['trip_duration']}-day trip to {preferences['destination']} with the following details:
        Budget: ${preferences['budget']}, Interests: {', '.join(preferences['interests'])},Travel Style**: {preferences['travel_style']},Dietary Restrictions**: {preferences['dietary_restrictions']},Trip Duration: {preferences['trip_duration']} days,Destination: {preferences['destination']},Weather: {weather_data},Destination Overview: Provide a brief 3-5 lines description of {preferences['destination']} including famous places to visit and key attractions.
        The itinerary should include activities for {preferences['trip_duration']} days with specific recommendations for morning, afternoon, and evening activities each day.
        Mention travel options from the origin to the destination.
        Additionally, include suggestions for restaurants, scenic locations, accomodations and things to do for the user with a {preferences['travel_style']} travel style.
        """

        # Tokenize the prompt for input into the model
        inputs = self.tokenizer(prompt, padding=True, return_tensors="pt").to(self.device)

        # Generate the output using the model
        outputs = self.model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id,
            max_new_tokens=10000,  # Max length of the response
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
        )

        # Decode the generated response
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Return the itinerary generated from the model
        return response.strip()  # Remove leading/trailing whitespace



# Agent IN ACTION

In [ ]:
def main():
    # Collect user preferences
    user_input = {
        "budget": 1000,
        "interests": ["beaches", "museums", "local culture"],
        "travel_style": "relaxed",
        "dietary_restrictions": "none",
        "trip_duration": 2,
        "destination": "Cancun",
        "origin": "New York"
    }

    # Initialize the ItineraryAgent with the LLM model pipeline
    itinerary_agent = ItineraryAgent()

    # Generate itinerary based on preferences
    itinerary = itinerary_agent.generate_itinerary(user_input)

    # Print the generated travel plan
    print("Generated Itinerary:")
    print(itinerary)


if __name__ == "__main__":
    main()

Generated Itinerary:
Prompt: Give a breif description of the Cancun.Maximum 2-5 lines.
        Generate a detailed itinerary for a 2-day trip to Cancun with the following details:
        Budget: $1000, Interests: beaches, museums, local culture,Travel Style**: relaxed,Dietary Restrictions**: none,Trip Duration: 2 days,Destination: Cancun,Weather: Clear +24°C,Destination Overview: Provide a brief 3-5 lines description of Cancun including famous places to visit and key attractions.
        The itinerary should include activities for 2 days with specific recommendations for morning, afternoon, and evening activities each day. 
        Mention travel options from the origin to the destination.
        Additionally, include suggestions for restaurants, scenic locations, accomodations and things to do for the user with a relaxed travel style.
        

        **Cancun**

Cancun is a popular tourist destination located on the Caribbean coast of Mexico. Known for its white-sand beaches, crys